## Library

In [ ]:
import pandas as pd
import random
import copy

## A

In [ ]:
graph_facebook = pd.read_csv('./Q4/facebook_combined.txt', sep=' ', header=None, names=['u', 'v'])

graph_facebook

,u,v
0,0,1
1,0,2
2,0,3
3,0,4
4,0,5
...,...,...
88229,4026,4030
88230,4027,4031
88231,4027,4032
88232,4027,4038


In [ ]:
def calculate_adjacency_matrix(graph):
    G = list(set(list(graph.u.unique()) + list(graph.v.unique())))
    num_nodes_G = len(G)

    am_G = [[0] * num_nodes_G for _ in range(num_nodes_G)]
    
    num_edges = 0
    for i in range(num_nodes_G):
        neighbors = list(graph[graph['u'] == G[i]].v) + list(graph[graph['v'] == G[i]].u)
        for j in range(num_nodes_G):
            if G[j] in neighbors:
                am_G[i][j] = 1
                am_G[j][i] = 1
                num_edges += 1

    return G, num_nodes_G, num_edges, am_G

In [ ]:
# Undirected
facebook, num_nodes_facebook, num_edges_facebook, am_facebook = calculate_adjacency_matrix(graph_facebook)
print(f'Number of nodes : {num_nodes_facebook}')
print(f'Number of edges : {num_edges_facebook}')

Number of nodes : 4039
Number of edges : 176468


In [ ]:
def adjacency_matrix_to_list(adj_matrix):
    num_vertices = len(adj_matrix)

    adj_list = {}

    for i in range(num_vertices):
        adj_list[i] = []
        for j in range(num_vertices):
            if adj_matrix[i][j] == 1:
                adj_list[i].append(j)

    return adj_list

## B

In [ ]:
def realization(adjacency_matrix, p):
    random.seed(42)
    realization_list = []

    for _ in range(10):
        graph = copy.deepcopy(adjacency_matrix)
        
        for i in range(len(graph)):
            for j in range(i + 1, len(graph[i])):
                if graph[i][j] == 1 and random.random() > p:
                    graph[i][j] = 0
                    graph[j][i] = 0

        realization_list.append(graph)
    
    return realization_list

In [ ]:
realization_list = realization(am_facebook, 0.1)

## C

In [ ]:
degrees = []
for graph in realization_list:
    degrees.append({i: sum(graph[i]) for i in range(len(graph))})

In [ ]:
def independent_cascade_model(graph, seed_set):
    active_nodes = set(seed_set.copy())
    new_nodes = set(seed_set)

    while new_nodes:
        current_node = new_nodes.pop()
        neighbors = set(graph.get(current_node, [])) - active_nodes
        for neighbor in neighbors:
            # edge = (current_node, neighbor)
            if neighbor in graph[current_node]:
                new_nodes.add(neighbor)
                active_nodes.add(neighbor)

    return len(active_nodes)

In [ ]:
al_realization = []
for graph in realization_list:
    al_realization.append(adjacency_matrix_to_list(graph))

In [ ]:
# First greedy
S = []
for iteration in range(8):
    m = 0
    k = 0
    for node in range(len(am_facebook)):
        if node not in S:
            d = 0
            for i in range(len(al_realization)):
                d += degrees[i][node]
            d = d / len(al_realization)
            if d > m:
                m = d
                k = node
    S += [k]
    f = 0
    for graph in al_realization:
        f += independent_cascade_model(graph, S)
    f = f / len(al_realization)
    print(f'Iteration {iteration} -> f = {f}, Nodes : {S}')

# 0.6s

Iteration 0 -> f = 2959.4, Nodes : [107]
Iteration 1 -> f = 2959.4, Nodes : [107, 1912]
Iteration 2 -> f = 2959.4, Nodes : [107, 1912, 1684]
Iteration 3 -> f = 2959.4, Nodes : [107, 1912, 1684, 3437]
Iteration 4 -> f = 2978.5, Nodes : [107, 1912, 1684, 3437, 0]
Iteration 5 -> f = 2978.5, Nodes : [107, 1912, 1684, 3437, 0, 2347]
Iteration 6 -> f = 2978.5, Nodes : [107, 1912, 1684, 3437, 0, 2347, 2543]
Iteration 7 -> f = 2978.5, Nodes : [107, 1912, 1684, 3437, 0, 2347, 2543, 1888]


In [ ]:
# Second greedy
S = []
for iteration in range(8):
    i = random.choice(range(len(am_facebook)))
    while i in S:
        i = random.choice(range(len(am_facebook)))
    
    S += [i]
    f = 0
    for graph in al_realization:
        f += independent_cascade_model(graph, S)
    f = f / len(al_realization)
    print(f'Iteration {iteration} -> f = {f}, Nodes : {S}')

# 0.2s

Iteration 0 -> f = 2959.4, Nodes : [3286]
Iteration 1 -> f = 2960.2, Nodes : [3286, 3676]
Iteration 2 -> f = 2960.5, Nodes : [3286, 3676, 2205]
Iteration 3 -> f = 2960.5, Nodes : [3286, 3676, 2205, 3362]
Iteration 4 -> f = 2961.7, Nodes : [3286, 3676, 2205, 3362, 3932]
Iteration 5 -> f = 2980.8, Nodes : [3286, 3676, 2205, 3362, 3932, 299]
Iteration 6 -> f = 2980.8, Nodes : [3286, 3676, 2205, 3362, 3932, 299, 3800]
Iteration 7 -> f = 2980.9, Nodes : [3286, 3676, 2205, 3362, 3932, 299, 3800, 1062]


## D

In [ ]:
# Greedy        S'
S = []
for iteration in range(8):
    values = {}
    for node in range(len(am_facebook)):
        if node not in S:
            f = 0
            for graph in al_realization:
                f += independent_cascade_model(graph, S + [node])
            values[node] = f / len(al_realization)

    node = sorted(values.items(), key=lambda item: item[1])[::-1][:1][0][0]
    S += [node]
    print(f'Iteration {iteration} -> f = {values[node]}, Nodes : {S}')

# 14m 14.5s

Iteration 0 -> f = 2959.4, Nodes : [3971]
Iteration 1 -> f = 3032.1, Nodes : [3971, 809]
Iteration 2 -> f = 3055.7, Nodes : [3971, 809, 343]
Iteration 3 -> f = 3069.7, Nodes : [3971, 809, 343, 3980]
Iteration 4 -> f = 3072.7, Nodes : [3971, 809, 343, 3980, 2699]
Iteration 5 -> f = 3075.6, Nodes : [3971, 809, 343, 3980, 2699, 3012]
Iteration 6 -> f = 3078.4, Nodes : [3971, 809, 343, 3980, 2699, 3012, 3894]
Iteration 7 -> f = 3081.2, Nodes : [3971, 809, 343, 3980, 2699, 3012, 3894, 822]


In [ ]:
# Greedy        S"
A = []
F = 0
for iteration in range(8):
    values = {}
    for node in range(len(am_facebook)):
        f_AUs = 0
        for graph in al_realization:
            f_AUs += independent_cascade_model(graph, A + [node])
        values[node] = (f_AUs / len(al_realization))  - F

    node = sorted(values.items(), key=lambda item: item[1])[::-1][:1][0][0]
    A += [node]
    F += values[node]
    print(f'Iteration {iteration} -> \nThe difference from the previous iteration = {values[node]} \nf = {F}, Nodes : {A}')

# 19m 40.4s

Iteration 0 -> 
The difference from the previous iteration = 2959.4 
f = 2959.4, Nodes : [3971]
Iteration 1 -> 
The difference from the previous iteration = 72.69999999999982 
f = 3032.1, Nodes : [3971, 809]
Iteration 2 -> 
The difference from the previous iteration = 23.59999999999991 
f = 3055.7, Nodes : [3971, 809, 343]
Iteration 3 -> 
The difference from the previous iteration = 14.0 
f = 3069.7, Nodes : [3971, 809, 343, 3980]
Iteration 4 -> 
The difference from the previous iteration = 3.0 
f = 3072.7, Nodes : [3971, 809, 343, 3980, 2699]
Iteration 5 -> 
The difference from the previous iteration = 2.900000000000091 
f = 3075.6, Nodes : [3971, 809, 343, 3980, 2699, 3012]
Iteration 6 -> 
The difference from the previous iteration = 2.800000000000182 
f = 3078.4, Nodes : [3971, 809, 343, 3980, 2699, 3012, 3894]
Iteration 7 -> 
The difference from the previous iteration = 2.799999999999727 
f = 3081.2, Nodes : [3971, 809, 343, 3980, 2699, 3012, 3894, 822]


In [ ]:
import heapq

class MinHeap:
    def __init__(self):
        self.heap = []

    def push(self, key_value_pair):
        heapq.heappush(self.heap, key_value_pair)

    def pop(self):
        if self.heap:
            return heapq.heappop(self.heap)
        else:
            raise IndexError("pop from an empty heap")

    def peek(self):
        if self.heap:
            return self.heap[0]
        else:
            return None

In [ ]:
# CELF

# S'
print('************** Greedy S\' **************')
S1 = []
F1 = 0
min_heap = MinHeap()
for iteration in range(8):
    if iteration == 0:
        for node in range(len(am_facebook)):
            f_AUs = 0 
            for graph in al_realization:
                f_AUs += independent_cascade_model(graph, [node])
            min_heap.push(((-1) * f_AUs / len(al_realization), node))
        m, node = min_heap.pop()
    else:
        while True:
            m, node = min_heap.pop()
            f_AUs = 0 
            for graph in al_realization:
                f_AUs += independent_cascade_model(graph, S1 + [node])
            f_AUs = (f_AUs / len(al_realization)) - F1
            if f_AUs >= ((-1) * min_heap.peek()[0]):
                m = (-1) * f_AUs
                break
            else:
                min_heap.push(((-1) * f_AUs, node))

    S1 += [node]
    F1 += (-1)*m
    print(f'Iteration {iteration} -> \nThe difference from the previous iteration = {(-1)*m} \nf = {F1}, Nodes : {S1}')

# S"
print('\n************** Greedy S\" **************')
S2 = []
F2 = 0
min_heap = MinHeap()
for iteration in range(8):
    if min_heap.peek() == None:
        for node in range(len(am_facebook)):
            f_AUs = 0 
            for graph in al_realization:
                f_AUs += independent_cascade_model(graph, [node])
            min_heap.push(((-1) * f_AUs / len(al_realization), node))
        m, node = min_heap.pop()
    else:
        while True:
            m, node = min_heap.pop()
            f_AUs = 0
            for graph in al_realization:
                f_AUs += independent_cascade_model(graph, S2 + [node])
            f_AUs = (f_AUs / len(al_realization)) - F2
            if f_AUs >= ((-1) * min_heap.peek()[0]):
                m = (-1) * f_AUs
                break
            else:
                min_heap.push(((-1) * f_AUs, node))

    S2 += [node]
    F2 += (-1)*m
    print(f'Iteration {iteration} -> \nThe difference from the previous iteration = {(-1)*m} \nf = {F2}, Nodes : {S2}')

print('\n************** CELF **************')
F = max(F1, F2)
S = S1 if F == F1 else S2
print(f'f(S) = {F}, Nodes : {S}')

# 8m 56.0s

************** Greedy S' **************
Iteration 0 -> 
The difference from the previous iteration = 2959.4 
f = 2959.4, Nodes : [107]
Iteration 1 -> 
The difference from the previous iteration = 72.69999999999982 
f = 3032.1, Nodes : [107, 773]
Iteration 2 -> 
The difference from the previous iteration = 23.59999999999991 
f = 3055.7, Nodes : [107, 773, 115]
Iteration 3 -> 
The difference from the previous iteration = 14.0 
f = 3069.7, Nodes : [107, 773, 115, 3980]
Iteration 4 -> 
The difference from the previous iteration = 3.0 
f = 3072.7, Nodes : [107, 773, 115, 3980, 2699]
Iteration 5 -> 
The difference from the previous iteration = 2.900000000000091 
f = 3075.6, Nodes : [107, 773, 115, 3980, 2699, 3012]
Iteration 6 -> 
The difference from the previous iteration = 2.800000000000182 
f = 3078.4, Nodes : [107, 773, 115, 3980, 2699, 3012, 822]
Iteration 7 -> 
The difference from the previous iteration = 2.799999999999727 
f = 3081.2, Nodes : [107, 773, 115, 3980, 2699, 3012, 822, 564